In [1]:
import glob
import IPython.parallel as P
import pandas as pd
import numpy as np

In [2]:
c = P.Client()

In [3]:
view = c.load_balanced_view()

In [4]:
%%px
import pandas as pd
import numpy as np

In [5]:
files = glob.glob('/RHESSys/ComoCreek_new/clim/smexperiment/hourly/*.snow_melt_input')

In [20]:
def computeSM(fl):
    num = int(fl.split('_')[-3].split('.')[0])

    data = pd.read_table(fl,delim_whitespace=True, names=['Y','M','D','h','SM'], skiprows=1)

    sm = data.SM.mean() # compute average snowmelt [m/day] over the time series
    sm_in = data.SM.sum() # total snowmelt 
    
    return num,sm,sm_in # output parameter number and snowmelt rate 

In [21]:
res = view.map(computeSM,files)

In [22]:
res.ready()

False

In [25]:
res.progress

29500

In [26]:
num,sm,sm_in = zip(*res)

In [27]:
num = np.array(num)
sm = np.array(sm)
sm_in = np.array(sm_in)

In [28]:
# merge w/ the smex1 df

In [29]:
smex1 = pd.read_pickle('./data/smex1.pcl')

In [30]:
tmp = pd.DataFrame()
for arr,key in zip([num,sm,sm_in],['idx','sm','sm_in']):
    tmp[key] = arr

In [31]:
smex1 = pd.merge(smex1,tmp,on='idx')

In [32]:
smex1.head()

,Tnosnow,TpeakSWE,peakSWE,SMR,ref,date_ONS,DOAS,date_AS,idx,MAP,ET,Q,Qbf,Qrf,rech,sat,sm,sm_in
0,223,172,0.201000,0.003941,2008,2008-05-10,15,2007-10-15,0,0.3567,383.90021,26.612353,26.612353,0,224.454184,0,0.003941,0.201000
1,223,172,0.207490,0.004068,2008,2008-05-10,15,2007-10-15,1,0.3567,383.90021,26.612353,26.612353,0,224.454184,0,0.004068,0.207490
2,223,172,0.213980,0.004196,2008,2008-05-10,15,2007-10-15,2,0.3567,383.90021,26.612353,26.612353,0,224.454184,0,0.004196,0.213980
3,223,172,0.220469,0.004323,2008,2008-05-10,15,2007-10-15,3,0.3567,383.90021,26.612353,26.612353,0,224.454184,0,0.004323,0.220469
4,223,172,0.226959,0.004450,2008,2008-05-10,15,2007-10-15,4,0.3567,383.90021,26.612353,26.612353,0,224.454184,0,0.004450,0.226959


In [17]:
np.savez_compressed('./data/computed_sm_smex1.npz',idx=num,computed_sm=sm)

In [33]:
tmp = smex1['peakSWE'] - smex1['sm_in']

In [34]:
np.sum(tmp)

2.9408167841471489e-06